# EDA (Exploratory Data Analysis) : 탐색적 데이터 분석
* 데이터가 들어왔을 때, 이를 다양한 각도에서 관찰하고 이해하는 과정입니다. 

* 한마디로 데이터를 분석하기 전에 그래프나 통계적인 방법으로 자료를 직관적으로 바라보는 과정

## EDA가 필요한 이유
* 데이터의 분포 및 값을 검토함으로써 데이터가 표현하는 현상을 더 잘 이해하고, 데이터에 대한 잠재적인 문제를 발견할 수 있습니다. 

  이를 통해, 본격적인 분석에 들어가기에 앞서 데이터의 수집을 결정할 수 있습니다.

* 다양한 각도에서 살펴보는 과정을 통해 문제 정의 단계에서 미쳐 발생하지 못했을 다양한 패턴을 발견하고, 

  이를 바탕으로 기존의 가설을 수정하거나 새로운 가설을 세울 수 있습니다.

# Kaggle 기본 예제
# 타이타닉 탑승자 데이터를 기반으로 생존자 예측해보기

## 필요한 모듈 import 및 데이터 로드

In [ ]:
from google.colab import files
files.upload()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
%matplotlib inline

titanic_df = pd.read_csv('titanic_train.csv')
titanic_df.head()

In [ ]:
# 데이터 정보 확인
titanic_df.info()

In [ ]:
'''
# 대략적인 컬럼 정보
 0   PassengerId  891 non-null    int64        -   탑승자 데이터 일련번호
 1   Survived     891 non-null    int64        -   생존여부 (0 : 사망, 1 : 생존)
 2   Pclass       891 non-null    int64        -   티켓의 선실 등급 (1 : 1등석, 2 : 2등석, 3 : 3등석)
 3   Name         891 non-null    object       -   탑승자 이름
 4   Sex          891 non-null    object       -   탑승자 성별
 5   Age          714 non-null    float64      -   탑승자 나이
 6   SibSp        891 non-null    int64        -   같이 탑승한 형제자매 또는 배우자 인원수
 7   Parch        891 non-null    int64        -   같이 탑승한 부모님 또는 어린이 인원수
 8   Ticket       891 non-null    object       -   티켓번호
 9   Fare         891 non-null    float64      -   요금
 10  Cabin        204 non-null    object       -   선실번호
 11  Embarked     889 non-null    object       -   중간 정착 항구(C : Cherbourg, Q : Queenstown, S : Southhampton)
'''

## 데이터 전처리

In [ ]:
# 기본적으로 머신러닝 알고리즘은 null을 허용하지 않으므로 null처리가 필요
# 현재 데이터 상태를 고려하여 채우는 방법을 선택 : fillna()

titanic_df['Age'].fillna(titanic_df['Age'].mean(), inplace=True)
titanic_df['Cabin'].fillna('N', inplace=True)
titanic_df['Embarked'].fillna('N', inplace=True)

In [ ]:
titanic_df.head()

In [ ]:
# null 처리가 잘 되었는지 확인
titanic_df.isnull().sum()

### 데이터 탐색

In [ ]:
# 문자열 데이터 확인
print('sex 분포 : \n', titanic_df['Sex'].value_counts())

In [ ]:
print('Cabin 분포 : \n', titanic_df['Cabin'].value_counts())

In [ ]:
print('Embarked 분포 : \n', titanic_df['Embarked'].value_counts())

In [ ]:
# Cabin 속성을 첫글자만 추출하여 정리
titanic_df['Cabin'] = titanic_df['Cabin'].str[:1]
titanic_df.Cabin.head()

In [ ]:
# 문자열 카테고리 피처를 숫자형 카테고리 피처로 변환 - 레이블 인코딩
from sklearn.preprocessing import LabelEncoder

# 한꺼번에 변환
def encode_features(df):
    fetures = ['Cabin', 'Sex', 'Embarked']
    for feature in fetures:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

In [ ]:
titanic_df = encode_features(titanic_df)
titanic_df.head()

### 데이터 탐색과 가설 설정

* 어떤 유형의 승객이 생존확률이 높았을지 확인 (가설)

  1) 성별
    
  2) 부자 또는 유명인

  3) 선실 등급

In [ ]:
# 1) 성별에 따른 생존 결과 확인 - 데이터 (0 : 사망, 1 : 생존) || male : 1, female : 0
titanic_df.groupby(['Sex', 'Survived'])['Survived'].count()

In [ ]:
# 시각화를 통해 직관적으로 판단할 수 있도록 도울 수 있음
sns.barplot(x='Sex', y='Survived', data=titanic_df)

In [ ]:
# 2) 부자 또는 유명인과 생존 상관관계
sns.barplot(x='Pclass', y='Survived', hue='Sex', data=titanic_df)

In [ ]:
# 나이에 따른 생존 상관관계
# 나이는 값 종류가 다양하므로 범위(범주)별로 구분해서 파악

def get_category(age):
    category = ''
    if age <= -1 : category = 'Unknown'
    elif age <= 5 : category = 'Baby'
    elif age <= 12 : category = 'Child'
    elif age <= 18 : category = 'Teenager'
    elif age <= 25 : category = 'Student'
    elif age <= 35 : category = 'Young Adult'
    elif age <= 60 : category = 'Adult'
    else : category = 'Elderly'
        
    return category

In [ ]:
# x축 names
group_names = ['Unknown', 'Baby', 'Child', 'Teenager', 'Student', 'Young Adult', 'Adult', 'Elderly']

titanic_df['Age_Category'] = titanic_df['Age'].apply(lambda x : get_category(x))

In [ ]:
# 막대 그래프 크기 조정
plt.figure(figsize=(10, 6))

sns.barplot(x='Age_Category', y='Survived', hue='Sex', data=titanic_df, order=group_names)

* 지금까지의 전처리 작업을 한번에 수행할 수 있도록 함수로 만들어두기

In [ ]:
from sklearn.preprocessing import LabelEncoder

# null 처리 함수
def fillna(df):
    df['Age'].fillna(titanic_df['Age'].mean(), inplace=True)
    df['Cabin'].fillna('N', inplace=True)
    df['Embarked'].fillna('N', inplace=True)
    return df

# 머신러닝 알고리즘에게 불필요한 속성 제거
def drop_features(df):
    df.drop(['PassengerId', 'Name', 'Ticket'], axis=1, inplace=True)
    return df

# 레이블 인코딩
def encoding_features(df):
    df['Cabin'] = df['Cabin'].str[:1]
    fetures = ['Cabin', 'Sex', 'Embarked']
    for feature in fetures:
        le = LabelEncoder()
        le = le.fit(df[feature])
        df[feature] = le.transform(df[feature])
    return df

# 최종 전처리 수행
def transform_features(df):
    df = fillna(df)
    df = drop_features(df)
    df = encoding_features(df)
    return df

* 새로 만든 함수를 이용해서 다시 원본 데이터를 최종 전처리 

In [ ]:
# 원본 데이터 로드
titanic_df = pd.read_csv('titanic_train.csv')

# 레이블 데이터 셋
y_titanic_df = titanic_df['Survived']
X_titanic_df = titanic_df.drop('Survived', axis=1)

# 학습할 피처 데이터 셋 - 전체 전처리
X_titanic_df = transform_features(X_titanic_df)

In [ ]:
X_titanic_df.head()

## 모델 학습

In [ ]:
# 학습 세트와 테스트 데이터 세트 분리
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X_titanic_df, y_titanic_df, test_size=0.2, random_state=11)

In [ ]:
# 예측 시 많이 사용하는 알고리즘 3가지를 통해 각각 예측하고 평가하기
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

dt_clf = DecisionTreeClassifier(random_state=11)
rf_clf = RandomForestClassifier(random_state=11)
lr_clf = LogisticRegression(random_state=11)

# 각 알고리즘 별로 학습/예측/평가
dt_clf.fit(X_train, y_train)
dt_pred = dt_clf.predict(X_test)
print('DecistionTree accuracy : {0:.4f}'.format(accuracy_score(y_test, dt_pred)))

rf_clf.fit(X_train, y_train)
rf_pred = rf_clf.predict(X_test)
print('RandomForest accuracy : {0:.4f}'.format(accuracy_score(y_test, rf_pred)))

lr_clf.fit(X_train, y_train)
lr_pred = lr_clf.predict(X_test)
print('LogisticRegression accuracy : {0:.4f}'.format(accuracy_score(y_test, lr_pred)))